In [319]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
import math
#from datetime import datetime

In [320]:
#Read the csv file
df = pd.read_csv('Data/GBIMEfinal.csv')
df.head() #7 columns, including the Date. 

,Unnamed: 0,t,c,o,h,l,v
0,0,2012-09-09 14:59:59,46.052427,46.052427,46.052427,42.894547,50
1,1,2012-09-10 14:59:59,50.526092,46.052427,50.526092,46.841898,254
2,2,2012-09-11 14:59:59,55.526070,50.526092,55.526070,51.315562,376
3,3,2012-09-12 14:59:59,61.052361,55.526070,61.052361,56.578697,1131
4,4,2012-09-13 14:59:59,67.104966,61.052361,67.104966,62.104988,27430


In [321]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [322]:
df['t'] = pd.to_datetime(df['t']).dt.date


In [323]:
df2 = df.copy()


In [324]:
df2.drop('t',axis=1,inplace=True)


In [377]:
df2.corr()

,c,o,h,l,v
c,1.000000,0.997744,0.998968,0.999034,0.551238
o,0.997744,1.000000,0.998792,0.998749,0.545597
h,0.998968,0.998792,1.000000,0.998593,0.557798
l,0.999034,0.998749,0.998593,1.000000,0.540675
v,0.551238,0.545597,0.557798,0.540675,1.000000


In [326]:
df

,t,c,o,h,l,v
0,2012-09-09,46.052427,46.052427,46.052427,42.894547,50
1,2012-09-10,50.526092,46.052427,50.526092,46.841898,254
2,2012-09-11,55.526070,50.526092,55.526070,51.315562,376
3,2012-09-12,61.052361,55.526070,61.052361,56.578697,1131
4,2012-09-13,67.104966,61.052361,67.104966,62.104988,27430
...,...,...,...,...,...,...
1992,2023-05-07,183.000000,187.000000,187.000000,183.000000,49157
1993,2023-05-08,181.500000,186.000000,186.000000,180.200000,70629
1994,2023-05-09,183.200000,184.900000,184.900000,180.400000,59202
1995,2023-05-10,183.200000,185.000000,185.000000,182.200000,30993


In [327]:
#Separate dates for future plotting
train_dates = pd.to_datetime(df['t'])
print(train_dates.tail(15)) #Check last few dates.

1982   2023-04-20
1983   2023-04-23
1984   2023-04-24
1985   2023-04-25
1986   2023-04-26
1987   2023-04-27
1988   2023-04-30
1989   2023-05-02
1990   2023-05-03
1991   2023-05-04
1992   2023-05-07
1993   2023-05-08
1994   2023-05-09
1995   2023-05-10
1996   2023-05-11
Name: t, dtype: datetime64[ns]


In [364]:
c = list(df)[1]
v = list(df)[-1]
cols =[c , v]
cols

['c', 'v']

In [365]:
df_for_training = df[cols].astype(float)
df_for_training

,c,v
0,46.052427,50.0
1,50.526092,254.0
2,55.526070,376.0
3,61.052361,1131.0
4,67.104966,27430.0
...,...,...
1992,183.000000,49157.0
1993,181.500000,70629.0
1994,183.200000,59202.0
1995,183.200000,30993.0


In [366]:
training_data_len = math.ceil(len(df_for_training)*0.8)

training_data_len

1598

In [367]:
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

#scale the data
# scaler = MinMaxScaler(feature_range=(0,1))
# df_for_training_scaled = scaler.fit_transform(df_for_training)


In [368]:
#training the dataset
train_data = df_for_training_scaled[0:,:]

In [374]:
trainX = []
trainY = []

In [370]:
for i in range(60, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])


[array([-2.35388136, -2.29209326, -2.22303599, -2.14670952, -2.06311387,
       -2.0449409 , -2.00859497, -1.990422  , -2.00859497, -2.0449409 ,
       -2.09219062, -2.08855603, -2.08855603, -2.09219062, -2.09945981,
       -2.1030944 , -2.10672899, -2.08855603, -2.08855603, -2.08128684,
       -2.07401765, -2.07401765, -2.05947928, -2.07038306, -2.07765224,
       -2.06674846, -2.07401765, -2.0449409 , -2.02313334, -2.00859497,
       -1.97951822, -1.94680688, -1.91409553, -1.91046094, -1.88501879,
       -1.86321123, -1.87048041, -1.92499932, -1.85230744, -1.83413448,
       -1.77961557, -1.74326964, -1.74690423, -1.7105583 , -1.68511614,
       -1.65967399, -1.60878968, -1.60878968, -1.64513561, -1.66694317,
       -1.60878968, -1.53609781, -1.52155943, -1.58334752, -1.57607833,
       -1.57244374, -1.55427077, -1.43069459, -1.39798325, -1.40525244])]
[-1.3979832500132705]

[array([-2.35388136, -2.29209326, -2.22303599, -2.14670952, -2.06311387,
       -2.0449409 , -2.00859497, -1.9

In [371]:
trainX, trainY = np.array(trainX), np.array(trainY)

In [372]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (1937, 60).
trainY shape == (1937,).


In [373]:
# model = Sequential()
# model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
# model.add(LSTM(32, activation='relu', return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(trainY.shape[1]))\

#build the lstm model

model = Sequential()
model.add(LSTM(units = 50, activation='relu', return_sequences=True,  input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dropout(0.2))


model.add(LSTM(units = 60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(Dense(trainY.shape[1]))

model.add(LSTM(units = 80, activation='tanh', return_sequences=True))
model.add(Dropout(0.4))
model.add(Dense(trainY.shape[1]))

model.add(LSTM(units = 120, activation='tanh'))
model.add(Dropout(0.5))


model.add(Dense(trainY.shape[1]))

IndexError: tuple index out of range

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.summary()


In [ ]:
history = model.fit(trainX, trainY, epochs=25, batch_size=25, validation_split=0.1, verbose=1)

In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()